In [0]:
from google.colab import files
!pip install -q kaggle
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 64 May 18 19:46 kaggle.json


In [0]:
!kaggle datasets list

In [0]:
!kaggle  datasets download -d laurentmih/aisegmentcom-matting-human-datasets

In [0]:
import tensorflow as tf
import keras
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from imutils import paths
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
from keras.applications import VGG16
# !unzip  '/content/aisegmentcom-matting-human-datasets.zip'

In [0]:
directory = '/content/clip_img'
new_directory = '/content/Images' 


def copy_images():
    for file_name in os.listdir(directory):
        sub_dir_path = directory + '/' + file_name
        if (os.path.isdir(sub_dir_path)):
          for file_name1 in os.listdir(sub_dir_path):
              sub_dir_path1 = sub_dir_path + '/' + file_name1            
              if (os.path.isdir(sub_dir_path1)):
                     for image_name in os.listdir(sub_dir_path1):
                         if image_name[-4:] == '.jpg':
                            img = cv2.imread(sub_dir_path1+'/'+image_name)                            
                            copied_image_path = new_directory + '/' + 'person_'+image_name
                            # print(copied_image_path)
                            if img.size !=0:
                                cv2.imwrite(copied_image_path, img)

copy_images()


In [0]:
# for image_name in os.listdir('/content/Images'):
                        
#                             img = '/content/Images'+'/'+image_name
#                             # print(img)                           
#                             os.remove(img)

In [0]:
imagePaths = list(paths.list_images('/content/Images'))
data = []
labels = []
 
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-1]
	image = load_img(imagePath, target_size=(224, 224),grayscale=True,color_mode='gray')#,grayscale=True,color_mode='gray'
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(0)
 
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32") #ndarray: data ndarray with shape (3548, 224, 224, 3)
# data=np.array(data).reshape(-1, 224, 224, 1)

labels = np.array(labels)   #ndarray: labels ndarray with shape (1, 3548)
# labels =(np.array(labels).reshape(1,-1))
# print(label)

In [0]:
labels =(np.array(labels).reshape(1,-1))

In [0]:
 
plt.imshow(data[1])
print(len(labels))
print(labels[2])


In [0]:
# data = []
# labels = []

In [0]:
 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [0]:
nsamples, nx, ny = data.shape
d2_train_dataset = data.reshape((nsamples,nx*ny))

In [0]:

 
(x_train, x_test, y_train, y_test) = train_test_split(data, labels,test_size=0.20, random_state=42)

In [0]:
print(len(y_train))
print(len(x_train))
print(x_train.shape)
print(y_train.shape)
plt.imshow(x_train[1])

In [0]:
# from keras.applications.resnet50 import ResNet50
# CLASS_COUNT = 10

# base_model = ResNet50(
#     weights='imagenet',
#     include_top=False, 
#     input_shape=(224, 224, 3), 
#     pooling='avg',
# )
# base_model.trainable = False
# model = tf.keras.models.Sequential([
#   base_model,
#   Dense(CLASS_COUNT, activation='softmax'),
# ])
model = keras.models.Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs = 5, validation_split=0.3)
# model.fit(x_train, y_train, batch_size = 50, epochs = 5, verbose = 1)

Train on 3246 samples, validate on 1392 samples
Epoch 1/5
3246/3246 [==============================] - 25s 8ms/step - loss: 0.0099 - accuracy: 0.9948 - val_loss: 5.0364e-07 - val_accuracy: 1.0000
Epoch 2/5
3246/3246 [==============================] - 17s 5ms/step - loss: 2.4033e-06 - accuracy: 1.0000 - val_loss: 8.5639e-11 - val_accuracy: 1.0000
Epoch 3/5
3246/3246 [==============================] - 17s 5ms/step - loss: 1.1708e-06 - accuracy: 1.0000 - val_loss: 8.5639e-11 - val_accuracy: 1.0000
Epoch 4/5
3246/3246 [==============================] - 17s 5ms/step - loss: 1.0785e-06 - accuracy: 1.0000 - val_loss: 2.2266e-09 - val_accuracy: 1.0000
Epoch 5/5
3246/3246 [==============================] - 17s 5ms/step - loss: 9.5188e-07 - accuracy: 1.0000 - val_loss: 7.1508e-08 - val_accuracy: 1.0000


In [0]:
 
x_train = tf.keras.utils.normalize(x_train, axis = 1)
y_train = np.array(y_train)
# x_train = tf.keras.utils.normalize(x_train, axis = 1)
x_test = tf.keras.utils.normalize(x_test,axis = 1)
# x_train = tf.keras.utils.normalize(x_train, axis = 1)
# x_test = tf.keras.utils.normalize(x_test,axis = 1)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()) 
model.add(tf.keras.layers.Dense(754,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(600,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax)) 
model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'] )
model.fit(x_train,y_train,batch_size=32,epochs = 20, validation_split=0.3)
predictions = model.predict(x_test)
val_loss, val_acc = model.evaluate(x_test, y_test)

In [0]:
y_train

array([0, 0, 0, ..., 0, 0, 0])